#                                              Shadow Cornerback + Coverage Analysis 🔒🏈🏆

![](https://jaguarswire.usatoday.com/wp-content/uploads/sites/46/2019/08/usatsi_11396149.jpg?resize=900,446)

The term **"Shadow Corner"** has been thrown around fantasy football leagues for some time now, and is a widely used indicator by savvy fantasy players when determining who to start/sit in their lineups each week.

So what actually is a **"Shadow Corner"**? 

It is generally the team's best cornerback on defense, being tasked with covering the opposing team's "best" wide receiver, for the majority of the game. This strategy has proven to be effective if your shadow corner has the ability to completely shut down the opposing team's best offensive weapon all game.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

![](https://images.actionnetwork.com/blog/2018/11/brad.gif)

You can see above, James Bradberry in the black jersey, attempted to shadow Mike Evans of the Buccaneers for a majority of the game in 2018 and was quite successful. He only allowed 1 catch for 16 yards.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Objective:

### 1. Identify Shadow Corners

### 2. Evaluate them

### 3. Identify certain characteristics about them

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Credits that inspired the code necessary for this analysis:
 1. @CPMP's Big Data Bowl 2020 initial wrangling & Voronoi areas notebook - for standardizing tracking data
 2. @Shahsquatch's Field Control Model & @robikscube's notebook inspired the play visualizations plot
 3. @AndikaRachman's Identifying Coverage Scheme code helped expedite my revised data, showing distance between players, that code can be found in my other notebook
 4. @tombliss's additional data provided - coverage scheme and targeted receviers
 
 The data used has already been preprocessed and revised in my [other notebook](https://www.kaggle.com/jdruzzi/revised-bdb-data) to save time and RAM. I recommend running the other notebook locally due to kaggles RAM constraints

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 1000)

from sklearn.neighbors import BallTree

#from BDBUtils.Utilities import CreateNFLData
from IPython.core.display import HTML
import time
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from datetime import datetime
import pytz
from IPython.display import HTML
import scipy.stats as stats
import matplotlib as mpl
from matplotlib import animation, rc
from matplotlib.patches import Rectangle, Arrow
import tensorflow as tf
#from BDBUtils.Utilities import CreateNFLData

import os
from pylab import rcParams
import warnings  
warnings.filterwarnings('ignore')




def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    
    return df


def get_dx_dy(radian_angle, dist):
    dx = dist * math.cos(radian_angle)
    dy = dist * math.sin(radian_angle)
    return dx, dy



def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12*2, 6.33*2)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='slategrey', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.3,
                                edgecolor='k',
                                facecolor='royalblue',
                                alpha=0.4,
                                zorder=1)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.3,
                                edgecolor='k',
                                facecolor='royalblue',
                                alpha=0.4,
                                zorder=1)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 230)
    plt.ylim(0, 93.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax


class CreateNFLData:

    def __init__(self):
        pass

    def LoadData(self, Normal=True):
        if Normal == True:
            print("Loading Original Data")
            globbed_files = glob.glob("../input/nfl-big-data-bowl-2021/week*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        
        else:
            print("Loading Modified Data")
            globbed_files = glob.glob("Revised Data/*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        return WeekData



    def Standardize(self,W):
        print("Standardizing Data..")
        W['Dir_rad'] = np.mod(90 - W.dir, 360) * math.pi/180.0
        W['ToLeft'] = W.playDirection == "left"
        W['TeamOnOffense'] = "home"
        W.loc[W.possessionTeam != W.PlayerTeam, 'TeamOnOffense'] = "away"
        W['IsOnOffense'] = W.PlayerTeam == W.TeamOnOffense # Is player on offense?
        W['YardLine_std'] = 100 - W.yardlineNumber
        W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'YardLine_std'
                ] = W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'yardlineNumber']
        W['X_std'] = W.x
        W.loc[W.ToLeft, 'X_std'] = 120 - W.loc[W.ToLeft, 'x'] 
        W['Y_std'] = W.y
        W.loc[W.ToLeft, 'Y_std'] = 160/3 - W.loc[W.ToLeft, 'y'] 
        #W['Orientation_std'] = -90 + W.Orientation
        #W.loc[W.ToLeft, 'Orientation_std'] = np.mod(180 + W.loc[W.ToLeft, 'Orientation_std'], 360)
        W['Dir_std'] = W.Dir_rad
        W.loc[W.ToLeft, 'Dir_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Dir_rad'], 2*np.pi)
        W['dx'] = round(W['s']*np.cos(W['Dir_std']),2)
        W['dy'] = round(W['s']*np.sin(W['Dir_std']),2)
        W['X_std'] = round(W['X_std'],2)
        W['Y_std'] = round(W['Y_std'],2)
        #W['Orientation_rad'] = np.mod(W.o, 360) * math.pi/180.0
        W['Orientation_rad'] = np.mod(-W.o + 90, 360) * math.pi/180.0
        W['Orientation_std'] = W.Orientation_rad
        W.loc[W.ToLeft, 'Orientation_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Orientation_rad'], 2*np.pi)
        W['MPH'] = W['s'] / 0.488889
        return W


    def mergeGameData2(self,df):
        print("Merging Data..")
        games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv', index_col=0)
        plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv', index_col=0, usecols= ['gameId', 'playId','yardlineNumber','possessionTeam','yardlineSide'])
        df = pd.merge(df, games, how='left', left_on=['gameId'], right_on=['gameId'])
        df['PlayerTeam'] = np.where(df['team'] == 'home', df['homeTeamAbbr'], df['visitorTeamAbbr'] )
        df = df.drop(['homeTeamAbbr','visitorTeamAbbr'], axis=1)
        df = pd.merge(df, plays, how='left', left_on=['gameId', 'playId'], right_on=['gameId', 'playId'])
        df['OnOffense'] = np.where(df['PlayerTeam'] == df['possessionTeam'], True, False)
        TargetedWR = pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/targetedReceiver.csv', index_col=0)
        Coverages = pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/coverages_week1.csv', index_col=0)
        df = pd.merge(df, Coverages, how='left', left_on=['gameId','playId'], right_on=['gameId','playId'])
        df = pd.merge(df, TargetedWR, how='left', left_on=['gameId','playId'], right_on=['gameId','playId'])
        Offense = df.query('OnOffense == 1').reset_index()
        Defense = df.query('OnOffense == 0').reset_index()
        Football = df.query('displayName == "Football"').reset_index()
        return df
    
    def PositionRank(self,df):
        print("Creating Position Ranks..")
        group = df.groupby(['week','gameId','possessionTeam','targetNflId','playId'])['offensePlayResult','epa_x','yac_epa','air_epa','air_yards','complete_pass','touchdown'].max().reset_index()
        group = group.groupby(['week','gameId','possessionTeam','targetNflId'])['offensePlayResult','epa_x','yac_epa','air_epa','air_yards','complete_pass','touchdown'].agg(['sum','count']).reset_index()
        group.columns = [' '.join(col).strip() for col in group.columns.values]
        group.columns = group.columns.str.replace('sum' , '')
        #wrs= df.filter(['gameId','targetNflId',"receiver_player_id","complete_pass","down","qb_dropback","air_yards","yards_after_catch",'yards_gained','shotgun','no_huddle','pass_touchdown','yardline_100','ydstogo','pass_length','pass_location','epa','air_epa','yac_epa'])
        #wrs['yards_after_catch'] = wrs['yards_after_catch'].fillna(0)

        #wrs['Targets'] = (wrs['yardline_100'] > 0).astype(int)

        Cols = ['week', 'gameId','possessionTeam','targetNflId','offensePlayResult count','complete_pass ','touchdown ', 'offensePlayResult ', 'air_yards ']
        group = group[Cols]
        group.columns = ['week', 'gameId','Team','targetNflId','Targets','Rec','TD', 'Yards ', 'Air_yards']

        group = group.drop_duplicates(subset=['week','gameId','targetNflId'])
        Numdf = ['Targets','Rec','TD', 'Yards ', 'Air_yards']
        for col in tqdm(Numdf):
            group[col + '_Rolling'] = group.groupby(['targetNflId']).apply(lambda x: x[col].shift(1).expanding(1).mean()).reset_index(0,drop=True)

        group['Targets_Rolling'] = group['Targets_Rolling'].fillna(group.groupby(['targetNflId'])['Targets_Rolling'].transform('median'))
        group['Rec_Rolling'] = group['Rec_Rolling'].fillna(group.groupby(['targetNflId'])['Rec_Rolling'].transform('median'))
        group['TD_Rolling'] = group['TD_Rolling'].fillna(group.groupby(['targetNflId'])['TD_Rolling'].transform('median'))
        group['Yards _Rolling'] = group['Yards _Rolling'].fillna(group.groupby(['targetNflId'])['Yards _Rolling'].transform('median'))
        group['Air_yards_Rolling'] = group['Air_yards_Rolling'].fillna(group.groupby(['targetNflId'])['Air_yards_Rolling'].transform('median'))
        players = pd.read_csv('players.csv', index_col=0, usecols=['nflId','position','displayName'])
        group = pd.merge(group, players, how='left',left_on=['targetNflId'],right_on=['nflId'])
        group['Pos_Score'] = group['Targets_Rolling']*.35	 + group['Yards _Rolling']*.30 + group['Air_yards_Rolling']*.20 + group['TD_Rolling']*.15
        group['Pos_Score'] = group['Pos_Score'].fillna(0)
        group['Pos_Rank'] = group['position'] + round(group.groupby(['Team','gameId','position'])['Pos_Score'].rank(ascending=False),0).astype(int).astype(str)

        Cols = ['gameId','targetNflId','Pos_Rank']
        group1 = group[Cols]


        Cols = [1,3,5,7,9,13]

        for c in Cols:
            trim1 = group[(group['Pos_Rank'] == "WR1") & (group['week'] == c) ].filter(['displayName','Team','Pos_Score','Pos_Rank'],axis=1).sort_values(by=['Pos_Score'], ascending=False).reset_index().head(5)
            trim2 = group[(group['Pos_Rank'] == "WR2") & (group['week'] == c) ].filter(['displayName','Team','Pos_Score','Pos_Rank'],axis=1).sort_values(by=['Pos_Score'], ascending=False).reset_index().head(5)
            print('Week ',str(c))
            display(HTML(trim1.head(5).to_html()))
            display(HTML(trim2.head(5).to_html()))
   #     group1.to_csv('Pos_Rank.csv')
        df1 = pd.merge(df, group1, how='left',left_on=['gameId','nflId'],right_on=['gameId','targetNflId'])
        return df1
    
    def FrameData(self,df):
        NotNone = df.query('event != "None"')
        NotNone = NotNone.groupby(['gameId','playId','event'])['frameId'].max().reset_index()
        NotNone = NotNone.set_index(['gameId','playId','event'], drop= True).unstack('event').reset_index()
        NotNone.columns = [' '.join(col).strip() for col in NotNone.columns.values]
        NotNone.columns = NotNone.columns.str.replace('frameId' , '')
        NotNone.columns = NotNone.columns.str.replace(' ' , '')
        NotNone = NotNone.filter(['gameId','playId','ball_snap', 'pass_arrived', 'pass_forward','penalty_flag', 'play_action','touchdown','pass_outcome_caught'], axis=1)
        for col in tqdm(NotNone.columns):
            NotNone['Contains_' + str(col)] = np.where(NotNone[col] > 0, True, False)


        WeekData1 = pd.merge(df, NotNone, how="left", left_on=['gameId','playId'], right_on=['gameId','playId'] )
        WeekData1['After_snap'] = np.where(WeekData1['frameId'] > WeekData1['ball_snap'],1,0)
        WeekData1['After_Throw'] = np.where(WeekData1['frameId'] > WeekData1['pass_forward'],1,0)
        WeekData1['After_PassArrived'] = np.where(WeekData1['frameId'] > WeekData1['pass_arrived'],1,0)
        WeekData1['After_PlayAction'] = np.where(WeekData1['frameId'] > WeekData1['play_action'],1,0)
   #     WeekData1['After_run_pass_option'] = np.where(WeekData1['frameId'] > WeekData1['run_pass_option'],1,0)
        WeekData1['After_Catch'] = np.where(WeekData1['frameId'] > WeekData1['pass_outcome_caught'],1,0)
        return WeekData1
    
    def import_data(self,file):
        """create a dataframe and optimize its memory usage"""
        df = pd.read_csv(file, low_memory=False)
        df = reduce_mem_usage(df)
        return df
    

    

In [ ]:
import gc

Create = CreateNFLData()

start = time.process_time()

Weeks = range(1,18)

#globbed_files = glob.glob("../input/revised-data/*.csv") #creates a list of all csv files
data = []
for n in tqdm(Weeks):
    filename = '../input/revised-data/week' + str(n) + '.csv'
    frame = Create.import_data(filename)
    frame = frame.drop(['gameDate', 'gameTimeEastern','playDirection','yardlineSide','ToLeft', 'Opp_Dist_COpp.1'], axis=1)
    data.append(frame)
    gc.collect()

WeekData = pd.concat(data).reset_index()
gc.collect()
del data
Finaldf = Create.FrameData(WeekData)
del WeekData
#Finaldf = Finaldf.drop(['ball_snap', 'field_goal_blocked', 'field_goal_fake', 'first_contact', 'fumble', 'fumble_defense_recovered', 'fumble_offense_recovered', 'handoff', 'huddle_break_offense', 'huddle_start_offense', 'lateral', 'line_set', 'man_in_motion', 'out_of_bounds', 'pass_arrived', 'pass_forward', 'pass_lateral', 'pass_outcome_caught', 'pass_outcome_incomplete', 'pass_outcome_interception', 'pass_outcome_touchdown', 'pass_shovel', 'pass_tipped', 'penalty_accepted', 'penalty_flag', 'play_action', 'punt_fake', 'qb_sack', 'qb_spike', 'qb_strip_sack', 'run', 'shift','tackle',  'touchback', 'touchdown'], axis=1)
Finaldf = Finaldf.drop(['ball_snap','pass_arrived',	'pass_forward',	'penalty_flag',	'play_action','Contains_gameId','IsOnOffense',	'Contains_playId',	'Contains_ball_snap',	'Contains_pass_arrived',	'Contains_pass_forward'], axis=1)
Finaldf['Code'] = Finaldf['gameId'].astype(str) + "-" + Finaldf['playId'].astype(str)


plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv', usecols=['gameId', 'playId','down', 'yardsToGo','penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI'])


plays['Code'] = plays['gameId'].astype(str) + "-" + plays['playId'].astype(str)
plays = plays.set_index('Code')
plays = plays.loc[~plays.index.duplicated(keep='first')]

Cols = ['down', 'yardsToGo','penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI']

for col in tqdm(Cols):
    Finaldf[col] = Finaldf.Code.map(plays[col])


#Finaldf = pd.merge(Finaldf,plays, how='left', left_on=['gameId', 'playId'], right_on=['gameId', 'playId'])

del plays
print((time.process_time() - start) / 60)
print(Finaldf.shape)
gc.collect()

In this revised dataset, I manipulated the data by standardizing all of the player tracking data to better help our analysis. I've also included some metrics like the euclidean distance between opposing teams closest player, as well as, players from the same team. Here are some key variables

###### COpp = Closest Opponent
###### CTm = Closest Teammate

In [ ]:
Trim = Finaldf.filter(['gameId','playId','frameId','nflId','displayName','jerseyNumber','position','X_std',	'Y_std','dx','dy','Orientation_std','MPH','a','Opp_Dist_COpp','closestOpp_Id','position_COpp','Pos_Rank_COpp','closestTeam_Id',	'Team_Dist_CTm','position_CTm','FootDist'],axis=1)
Trim.head(10)

# Position Rankings:

Position Rankings were done offline locally and added in the other notebook, WR1 = Teams best receiver, WR2 = 2nd best, ect.

Data was pulled from [2018 PBP data](https://github.com/guga31bb/nflfastR-data/blob/master/data/play_by_play_2018.csv.gz?)

Determing the best wide receiver for each team,on any given week can be quite subjective for some teams. Depending on injuries, and weekly inconsistencies from some players, it is diifcult to determine who is the "1st best", "2nd best" at each position.

I derived a simple formula based on a rolling mean lag of targets, yards, air yards, and touchdowns. Weights were determined from my personal domain knowlege.

Position Score = ['Targets_Rolling']*.35 + ['Yards _Rolling']*.30 + ['Air_yards_Rolling']*.20 + ['TD_Rolling']*.15

and then ranked from largest score to least, grouped by week,team,positon 

Position Rank = Position + (Position Score Rank)


# Alignment:

We also want to determine the position align of the corner at the beginning of the snap. Instead of taking the min/max Y coordinates of the corner to determine alignment, we will take the min/max of the wide receivers for each play and match that with the closest defender. Corners aligned with the 2 most outside receivers will be "Outside" and "Slot" if aligned with slot receivers.

# Determining a Shadow coverage snap:

My assumptions were that if a corner was lined up opposite of the receiver at the start of each play, then that play can be classified as a shadow coverage snap for the corner, regardless of whether it was Man or Zone coverage. In order to be classified as a true shadow coverage game, then the corner must have shadowed the receiver for more than 60% of the coverage snaps and must be more than 20 total snaps to qualify. 

In [ ]:
#Find the 2 most outside recievers by taking the max & min of the Y axis for each play
OutsideWR = Finaldf.query('event == "ball_snap" & OnOffense == True ').groupby(['gameId','playId'])['Y_std'].agg(['min','max']).reset_index()

Shadow = Finaldf.query('position == "CB" & event == "ball_snap"').groupby(['gameId','displayName','Pos_Rank_COpp'])['X_std'].count().reset_index()
Shadow = Shadow.pivot_table(index=['gameId','displayName'], columns='Pos_Rank_COpp', values='X_std',aggfunc=np.sum, fill_value=0, margins=True).reset_index()

cols = Shadow.drop(['gameId','displayName','All'], axis=1).columns
Shadow[cols] = Shadow[cols].div(Shadow[cols].sum(axis=1), axis=0)
Shadow = Shadow.query('gameId != "All"')

Shadow['WR1Shadow?'] = np.where((Shadow['WR1'] > .60) & (Shadow['All'] > 20), 1,0)
Shadow['WR2Shadow?'] = np.where((Shadow['WR2'] > .60) & (Shadow['All'] > 20), 1,0)
Shadow['WR3Shadow?'] = np.where((Shadow['WR3'] > .60) & (Shadow['All'] > 20), 1,0)
Shadow['TE1Shadow?'] = np.where((Shadow['TE1'] > .60) & (Shadow['All'] > 20), 1,0)
Shadow = Shadow.groupby(['gameId','displayName'])['WR1','WR1Shadow?','WR2','WR2Shadow?','WR3','WR3Shadow?','TE1','TE1Shadow?','All'].max().reset_index()
Shadow['gameId'] = Shadow['gameId'].astype(float)

del Trim
gc.collect()

Shadow.query('displayName == "Jalen Ramsey"')

#### How to read this:

Jalen Ramsey plays an average of 27 Ouside snaps per game, 7 Slot snaps, Guards the WR1 for the opposing team on 48% of routes, and had a true "shadow" coverage game with the WR1 nearly 31% of the time, and so on for WR2, WR3 and TE1.

In [ ]:
CBalign = Finaldf.query('position == "CB" & event == "ball_snap"').groupby(['gameId','playId','Y_std_COpp','displayName'])['Y_std','FootDist'].mean().reset_index()
CBalign1 = pd.merge(CBalign, OutsideWR, how='left', left_on=['gameId','playId'], right_on=['gameId','playId'])
CBalign1['Outside'] = np.where((CBalign1['Y_std_COpp'] == CBalign1['min']) | (CBalign1['Y_std_COpp'] == CBalign1['max']),1,0)
CBalign1['Slot'] = np.where(CBalign1['Outside'] == 0,1,0)

del OutsideWR
gc.collect()

Player = CBalign1.groupby(['displayName','gameId'])['Outside','Slot'].sum().reset_index()
Player['gameId'] = Player['gameId'].astype(float)
Player = pd.merge(Player, Shadow, how='left', left_on=['displayName',	'gameId'], right_on=['displayName',	'gameId'])
Player = Player.groupby(['displayName'])['Outside','Slot','WR1','WR1Shadow?','WR2','WR2Shadow?','WR3','WR3Shadow?','TE1','TE1Shadow?','All'].mean().reset_index()

del Shadow
gc.collect()

Player.query('displayName == "Jalen Ramsey"')

Here are the corners that generally shadow the WR1

In [ ]:
CBalign = Finaldf.query('position == "CB" & event == "ball_snap"').groupby(['displayName'])['Y_std','FootDist','Opp_Dist_COpp'].agg(['mean','std']).reset_index()
CBalign.columns = [' '.join(col).strip() for col in CBalign.columns.values]
CBalign.columns = CBalign.columns.str.replace('mean' , '')
CBalign.columns = CBalign.columns.str.replace(' ' , '')
CBalign = pd.merge(CBalign, Player, how='left', left_on=['displayName'], right_on=['displayName'])
CBalign['Total'] = CBalign['Outside'] + CBalign['Slot']
CBalign['OutsidePct'] = CBalign['Outside'] / CBalign['Total']
CBalign['SlotPct'] = CBalign['Slot'] / CBalign['Total']
del Player
gc.collect()
CBalign.query('All > 20 ').filter(['displayName','All','OutsidePct','WR1','WR1Shadow?','WR2','WR2Shadow?','WR3','WR3Shadow?'],axis=1).sort_values(by=['WR1Shadow?'], ascending=False).head(10)

# Separation:

We will take a look into how the corner's euclidean distance (Separation) from the closest offensive player fares over all of the events(snap of the ball, play action, etc.) and for each route in the route tree given in the dataset(Slant, Hitch, etc.), the distance metric will be called "Opp_Dist_COpp"

Well in order to give a fair evaluation, we would need to determine if the defensive playcall is zone or man. Zone coverage can skew separation metrics, due to the defender doing their job and staying a certain zone on the field, so we will have to determine man coverage. I did this by taking into account every frame the defender stayed with the closest offensive player. My assumptions were if they stayed with the recevier for more 60% of the frames, then it can be classified as true man-to-man coverage snap. Obviously, there are flaws to this logic, but for time and energy sake this should suffice. I may delve into predicting whether it's man / zone in another submission, and build off of this [academic paper](https://arxiv.org/pdf/1906.11373.pdf)

# EPA:

In addition to separation metrics, I will also track Expected Points Allowed (EPA) for each situation. I will not going into the specifics of EPA, but please refer to [this](https://www.stat.cmu.edu/~ryurko/files/greatlakes_2017.pdf) if you're interested. EPA can be used as an evaluation metric, "estimates a play’s value based on the change in situation, providing a point value". For offensive players a high EPA is good, for defensive players a low EPA is good.

In [ ]:
trimmed = Finaldf.query('After_snap == 1 & After_PassArrived == 0 & OnOffense == False')

Base = trimmed.groupby(['gameId','playId','nflId','displayName','position','PlayerTeam','closestOpp_Id','Pos_Rank_COpp','possessionTeam','passResult','offensePlayResult','isDefensivePI','epa','Targeted'])['Opp_Dist_COpp'].agg(['first','last','var','mean','std','median','count']).reset_index()
Base['Total'] = Base.groupby(['gameId','playId','displayName'])['count'].transform('sum')
Base['FramePct'] = Base['count'] / Base['Total']
Base['Coverage'] = np.where(Base['FramePct'] > .65,"Man-to-Man",'Zone')
Base['mean'] = Base['mean'].astype(np.float32)

Routes = trimmed.groupby(['gameId','playId','displayName','position','closestOpp_Id','route_COpp'])['Opp_Dist_COpp','epa'].agg(['mean']).reset_index()
Routes[('Opp_Dist_COpp', 'mean')] = Routes[('Opp_Dist_COpp', 'mean')].astype(np.float32)
Routes = Routes.set_index(['gameId','playId','displayName','position','closestOpp_Id','route_COpp'], drop= True).unstack('route_COpp').reset_index()
Routes.columns = [' '.join(col).strip() for col in Routes.columns.values]
Routes.columns = Routes.columns.str.replace('mean' , '')
Routes.columns = Routes.columns.str.replace(' ' , '')

Events = trimmed.groupby(['gameId','playId','displayName','position','closestOpp_Id','event'])['Opp_Dist_COpp','epa'].agg(['mean']).reset_index()
Events[('Opp_Dist_COpp', 'mean')] = Events[('Opp_Dist_COpp', 'mean')].astype(np.float32)
Events = Events.set_index(['gameId','playId','displayName','position','closestOpp_Id','event'], drop= True).unstack('event').reset_index()
Events.columns = [' '.join(col).strip() for col in Events.columns.values]
Events.columns = Events.columns.str.replace('mean' , '')
Events.columns = Events.columns.str.replace(' ' , '')

Comb = pd.merge(Base, Events, how='left', left_on=['gameId','playId','displayName','position','closestOpp_Id'], right_on=['gameId','playId','displayName','position','closestOpp_Id'])

Comb = pd.merge(Comb, Routes, how='left', left_on=['gameId','playId','displayName','position','closestOpp_Id'], right_on=['gameId','playId','displayName','position','closestOpp_Id'])

del Routes
del Events
del trimmed
gc.collect()

Comb

# Trim Data:

We want to trim this dataset down to:

- Man-to-Man coverage
- Defender was targeted (Target data was used from @tombliss's notebook, and matched with closest offensive player ID)
- Penalty = False

# Ground Covered:

I've also added a "Ground Covered" variable that takes the defenders separation with closest offensive player when the ball is leaving the Quarterback's hands, and takes the difference from the defenders separation when the pass arrives

A large ground covered value can show how quickly the defender is able to hone in on the receiver if he's been beat off the ball. But, then again a small value can make a case for them sticking closely with the receiver for the entirety of the play.

In [ ]:
Comb['Targeted'] = np.where(Comb['Targeted'] == True, 1, 0 )
Comb['Complete'] = np.where(Comb['passResult'] == "C", 1, 0 )
Comb['GroundCovered'] = Comb['Opp_Dist_COpppass_forward'] - Comb['Opp_Dist_COpppass_arrived']

![](https://wp.usatodaysports.com/wp-content/uploads/sites/76/2015/12/antwon-blake.gif)

Here you can see AJ Green blowing right by cornerback Antwon Blake, he simply was not able to make up the ground necessary to stop Green from making the catch

In [ ]:
Man = Comb.query('Coverage == "Man-to-Man" & position == "CB" & Targeted == 1 & isDefensivePI == False')
#Man = Man.groupby(['Defender','Defender_pos'])['mean','GO','HITCH','ANGLE','CORNER','CROSS','IN','OUT','POST','SCREEN','SLANT','WHEEL'].agg(['mean','count']).reset_index()
#Man = Man.groupby(['Defender','Defender_pos'])['mean','Targeted','Complete','offensePlayResult'].agg(['mean','count']).reset_index()
#Man.columns = Man.columns.map('_'.join)
#Man = Man.sort_values(by=['mean_count'], ascending=False).head(100).sort_values(by=['Complete_mean'], ascending=True)
#Man
Man = Man.groupby(['nflId','displayName','position']).agg({'mean':[('Avg_Separation', 'mean'),('Num_of_Routes', 'count')],
                                                            'Complete':[('Completions_Allowed', 'sum')],
                                                            'offensePlayResult':[('Yards_Allowed', 'sum'),('AvgYards_Allowed', 'mean')],
                                                            'GroundCovered':[('GroundCovered_Avg', 'mean')],
                                                            'epa':[('EPA', 'mean')],
                                                            'Opp_Dist_COppGO':[('Go_Separation', 'mean'),('Go_Routes', 'count')],
                                                            'Opp_Dist_COppHITCH':[('HITCH_Separation', 'mean'),('HITCH_Routes', 'count')],
                                                            'Opp_Dist_COppCROSS':[('CROSS_Separation', 'mean'),('CROSS_Routes', 'count')],
                                                            'Opp_Dist_COppOUT':[('OUT_Separation', 'mean'),('OUT_Routes', 'count')],
                                                            'Opp_Dist_COppPOST':[('POST_Separation', 'mean'),('POST_Routes', 'count')],
                                                            'Opp_Dist_COppIN':[('IN_Separation', 'mean'),('IN_Routes', 'count')],
                                                            'Opp_Dist_COppFLAT':[('FLAT_Separation', 'mean'),('FLAT_Routes', 'count')],
                                                            'Opp_Dist_COppCORNER':[('CORNER_Separation', 'mean'),('CORNER_Routes', 'count')],
                                                            'Opp_Dist_COppSLANT':[('SLANT_Separation', 'mean'),('SLANT_Routes', 'count')],
                                                            'epaGO':[('EPA_GO', 'mean')],
                                                            'epaHITCH':[('EPA_HITCH', 'mean')],
                                                            'epaCROSS':[('EPA_CROSS', 'mean')],
                                                            'epaOUT':[('EPA_OUT', 'mean')],
                                                            'epaPOST':[('EPA_POST', 'mean')],
                                                            'epaIN':[('EPA_IN', 'mean')],
                                                            'epaFLAT':[('EPA_FLAT', 'mean')],
                                                            'epaCORNER':[('EPA_CORNER', 'mean')],
                                                            'epaSLANT':[('EPA_SLANT', 'mean')],}).reset_index(drop=False)
Man.columns = Man.columns.map('_'.join)
Man['Comp_pct'] = Man['Complete_Completions_Allowed'] / Man['mean_Num_of_Routes']
Man = Man.sort_values(by=['mean_Num_of_Routes'], ascending=False).head(100).sort_values(by=['epa_EPA'], ascending=True)

Final = pd.merge(Man, CBalign, how='left', left_on=['displayName_'], right_on=['displayName'])
del Man
del Comb
gc.collect()
Final.head(10)

# Clustering Analysis:

We will cluster the cornerbacks based on their alignment to appropriately evaluate them. Outside cornerbacks are generally the best cornerbacks for their team and have a different playing style than a slot corner. Slot corners are lined up predominately opposite the inside recevier, and are usually quicker than most outside corners

So we will create 3 groups: Outside corners, Slot corners, and Hybrids (play slot and outside)


In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
Numdf = Final.select_dtypes(include=numerics)
#Numdf = Numdf.drop(['GameID'], axis=1)
Cols = ['Outside', 'Slot']
Numdf = Numdf[Cols]
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

x = SimpleImputer().fit_transform(Numdf)
x = StandardScaler().fit_transform(x)

import scipy.cluster.hierarchy as sch

dendrogram = sch.dendrogram(sch.linkage(x, method = 'ward'))
plt.title('Dendrogam', fontsize = 20)
plt.xlabel('Defenders')
plt.ylabel('Ecuclidean Distance')
plt.show()

The dendrogram confirms our theory

In [ ]:
# for basic mathematics operation 
import numpy as np
import pandas as pd
from pandas import plotting

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# for interactive visualizations
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected = True)
import plotly.figure_factory as ff

from sklearn.cluster import KMeans
#x = CBalign.drop(['Defender'], axis=1).values

from sklearn.cluster import KMeans
x = Final[['Outside', 'Slot','WR1Shadow?']].values
km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 1000, n_init = 10, random_state = 0)
km.fit(x)
labels = km.labels_
centroids = km.cluster_centers_

Final['labels'] =  labels
trace1 = go.Scatter3d(surfacecolor='darkgrey',
    x= Final['Outside'],
    y= Final['Slot'],
    z= Final['WR1Shadow?'],
    mode='markers',text=Final['displayName'],
     marker=dict(
        color = Final['labels'], 
        colorscale='Jet',
        size= 5,
        line=dict(
            color= Final['labels'],
            width= 10
        ),
        opacity=0.8
     )
)
df = [trace1]

layout = go.Layout(
    title = 'Corner Types',
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0  
    ),
    scene = dict(
            xaxis = dict(title  = 'Outside'),
            yaxis = dict(title  = 'Slot'),
            zaxis = dict(title  = 'WR1 Shadow %')
        )
)

fig = go.Figure(data = df, layout = layout)
py.iplot(fig)

Here you can visualize in 3D the cornerback clusters, with a Z dimension of WR1 shadow %

In [ ]:
Cols = ['nflId_', 'displayName_','labels','mean_Avg_Separation', 'mean_Num_of_Routes', 'epa_EPA', 'Comp_pct','offensePlayResult_AvgYards_Allowed', 'GroundCovered_GroundCovered_Avg','epaGO_EPA_GO', 'Opp_Dist_COppGO_Go_Separation', 'Opp_Dist_COppGO_Go_Routes', 'epaHITCH_EPA_HITCH','Opp_Dist_COppHITCH_HITCH_Separation', 'Opp_Dist_COppHITCH_HITCH_Routes', 'epaCROSS_EPA_CROSS',  'Opp_Dist_COppCROSS_CROSS_Separation', 'Opp_Dist_COppCROSS_CROSS_Routes', 'epaOUT_EPA_OUT', 'Opp_Dist_COppOUT_OUT_Separation', 'Opp_Dist_COppOUT_OUT_Routes', 'epaPOST_EPA_POST', 'Opp_Dist_COppPOST_POST_Separation', 'Opp_Dist_COppPOST_POST_Routes', 'epaIN_EPA_IN', 'Opp_Dist_COppIN_IN_Separation', 'Opp_Dist_COppIN_IN_Routes', 'epaFLAT_EPA_FLAT', 'Opp_Dist_COppFLAT_FLAT_Separation', 'Opp_Dist_COppFLAT_FLAT_Routes', 'epaCORNER_EPA_CORNER',  'Opp_Dist_COppCORNER_CORNER_Separation', 'Opp_Dist_COppCORNER_CORNER_Routes','epaSLANT_EPA_SLANT', 'Opp_Dist_COppSLANT_SLANT_Separation', 'Opp_Dist_COppSLANT_SLANT_Routes','Y_std',	'Y_stdstd',	'FootDist',	'FootDiststd',	'Opp_Dist_COpp',	'Opp_Dist_COppstd',	'Outside',	'Slot',	'WR1',	'WR1Shadow?',	'WR2',	'WR2Shadow?',	'WR3',	'WR3Shadow?',	'TE1',	'TE1Shadow?',	'All',	'Total',	'OutsidePct',	'SlotPct']
Final2 = Final[Cols]

del Final
gc.collect()

Final2.columns = ['nflId', 'Name','Cluster','Avg_Separation', 'Num_of_Routes', 'Avg_EPA', 'Comp_pct', 'AvgYards_Allowed', 'GroundCovered_Avg','EPA_GO', 'Go_Separation', 'Go_Routes', 'EPA_HITCH','HITCH_Separation', 'HITCH_Routes', 'EPA_CROSS',  'CROSS_Separation', 'CROSS_Routes', 'EPA_OUT', 'OUT_Separation', 'OUT_Routes', 'EPA_POST', 'POST_Separation', 'POST_Routes', 'EPA_IN', 'IN_Separation', 'IN_Routes', 'EPA_FLAT', 'FLAT_Separation', 'FLAT_Routes', 'EPA_CORNER',  'CORNER_Separation', 'CORNER_Routes','EPA_SLANT', 'SLANT_Separation', 'SLANT_Routes','Avg_Yaxis',	'Yaxis_std','AvgFootball_DistAtSnap',	'Football_DistAtSnap_std',	'AvgOpp_Dist_COpp_AtSnap',	'Opp_Dist_COppstd_AtSnap',	'Outside',	'Slot',	'WR1',	'WR1Shadow?',	'WR2',	'WR2Shadow?',	'WR3',	'WR3Shadow?',	'TE1',	'TE1Shadow?',	'All',	'Total',	'OutsidePct',	'SlotPct']
Final2.query('Cluster == 0').head(3)

# Ranking System:

Instead of taking these metrics at face value, let's rank how well the defender did for each variable compared to others in their cluster. Then we average the rankings across the board to get a single score value to determine who was really the best at their respective cluster positions.

The lower the score, the better!

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
Numdf = Final2.select_dtypes(include=numerics)
Cols = ['Avg_Separation', 'Avg_EPA', 'Comp_pct', 'AvgYards_Allowed', 'GroundCovered_Avg', 'EPA_GO', 'Go_Separation', 'EPA_HITCH', 'HITCH_Separation', 'EPA_CROSS', 'CROSS_Separation', 'EPA_OUT', 'OUT_Separation','EPA_POST', 'POST_Separation', 'EPA_IN', 'IN_Separation', 'EPA_FLAT', 'FLAT_Separation', 'EPA_CORNER', 'CORNER_Separation','EPA_SLANT', 'SLANT_Separation']
df1 = Final2.filter(['nflId','Name','Cluster'],axis=1)

for col in Cols:
    df1[col + '_rank'] = Final2.groupby(['Cluster'])[col].rank(ascending=True)

df1['Score'] = df1.iloc[:,-len(Cols):].mean(axis = 1, skipna = True)
Rankings = df1.filter(['nflId','Score'],axis=1).sort_values(by='Score',ascending=True)
Rankings = Rankings.set_index('nflId')
Rankings = Rankings.loc[~Rankings.index.duplicated(keep='first')]
Final2["Rank"] = Final2.nflId.map(Rankings['Score'])


display(HTML(Final2.head(5).to_html()))

display(HTML(df1.head(5).to_html()))
#df1.query('Name == "Josh Norman"')

# Top Corners By Cluster

## Outside Corners

In [ ]:
display(HTML(df1.query('Cluster == 0').filter(['Name','Score'],axis=1).sort_values(by='Score',ascending=True).head(10).to_html()))

## Slot Corners

In [ ]:
display(HTML(df1.query('Cluster == 1').filter(['Name','Score'],axis=1).sort_values(by='Score',ascending=True).head(10).to_html()))

## Hybrid Corners

In [ ]:
display(HTML(df1.query('Cluster == 2').filter(['Name','Score'],axis=1).sort_values(by='Score',ascending=True).head(10).to_html()))

Since outside corners are generally the best of the bunch, I'm crowning **Marlon Humphrey** as the best corner in 2018

# The Fairest Shadow Corner of them all in 2018: 


![](https://www.pennlive.com/resizer/kwEG97BhW1r6R3M8K_fQakZh-Jo=/1280x0/smart/cloudfront-us-east-1.images.arcpublishing.com/advancelocal/6VQZXVVQQRDFFK6MA32VUSX4OQ.jpg)

# Marlon Humphrey

### - Average EPA Cluster Rank: 1st 🥇

### - Average Completion Percent Cluster Rank: 1st 🥇

### - Average Separation Cluster Rank: 2nd 🥈

### - Average Yards Allowed Cluster Rank: 3rd 🥉

# Separation Vs. Routes:

Let's evaluate top cornerbacks and their separation/EPA ranks across the route tree, some defenders may struggle to maintain close separation depending on the route.

![](https://nfl-static.s3.amazonaws.com/attachments/ckdhpvebs087j0iodyvyrb7b6-nfl-flag-play-routetree1-1.44.106.986.571.max.png)

Here's a decent image I was able to find that showed some of the routes that a recevier runs.

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

Top10 = df1.query('Cluster == 0').sort_values(by='Score',ascending=True).head(10)

palette = cycle(px.colors.qualitative.Bold)
palette = cycle(px.colors.sequential.dense)

fig = go.Figure(data=[
    go.Bar(name='GO', x=Top10.Name, y=Top10.Go_Separation_rank,marker_color=next(palette)),
    go.Bar(name='Hitch', x=Top10.Name, y=Top10.HITCH_Separation_rank,marker_color=next(palette)),
    go.Bar(name='CROSS', x=Top10.Name, y=Top10.CROSS_Separation_rank,marker_color=next(palette)),
    go.Bar(name='OUT', x=Top10.Name, y=Top10.OUT_Separation_rank,marker_color=next(palette)),
    go.Bar(name='POST', x=Top10.Name, y=Top10.POST_Separation_rank,marker_color=next(palette)),
    go.Bar(name='IN', x=Top10.Name, y=Top10.IN_Separation_rank,marker_color=next(palette)),
    go.Bar(name='CORNER', x=Top10.Name, y=Top10.FLAT_Separation_rank,marker_color=next(palette)),
    go.Bar(name='FLAT', x=Top10.Name, y=Top10.CORNER_Separation_rank,marker_color=next(palette)),
    go.Bar(name='SLANT', x=Top10.Name, y=Top10.SLANT_Separation_rank,marker_color=next(palette)),

])
# Change the bar mode
fig.update_layout(barmode='group', template='plotly_dark', title="Avg. Separation Rank By Route")
fig.update_layout(
    autosize=False,
    width=1200,
    height=500,
)
fig.show()

You can see that Humphrey rarely let's his guy get a step on him, on nearly every route.

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

palette = cycle(px.colors.qualitative.Bold)
palette = cycle(px.colors.diverging.curl)
#palette = cycle(px.colors.cyclical.Edge)
#palette = cycle(px.colors.sequential.tempo)

Top10 = df1.query('Cluster == 0').sort_values(by='Score',ascending=True).head(10)

fig = go.Figure(data=[
    go.Bar(name='GO', x=Top10.Name, y=Top10.EPA_GO_rank,marker_color=next(palette)),
    go.Bar(name='Hitch', x=Top10.Name, y=Top10.EPA_HITCH_rank,marker_color=next(palette)),
    go.Bar(name='CROSS', x=Top10.Name, y=Top10.EPA_CROSS_rank,marker_color=next(palette)),
    go.Bar(name='OUT', x=Top10.Name, y=Top10.EPA_OUT_rank,marker_color=next(palette)),
    go.Bar(name='POST', x=Top10.Name, y=Top10.EPA_POST_rank,marker_color=next(palette)),
    go.Bar(name='IN', x=Top10.Name, y=Top10.EPA_IN_rank,marker_color=next(palette)),
    go.Bar(name='CORNER', x=Top10.Name, y=Top10.EPA_CORNER_rank,marker_color=next(palette)),
    go.Bar(name='FLAT', x=Top10.Name, y=Top10.EPA_FLAT_rank,marker_color=next(palette)),
    go.Bar(name='SLANT', x=Top10.Name, y=Top10.EPA_SLANT_rank,marker_color=next(palette)),

])
# Change the bar mode
fig.update_layout(barmode='group', template='plotly_dark', title="Avg. EPA Rank By Route")
fig.update_layout(
    autosize=False,
    width=1200,
    height=500,
)
fig.show()

Humphrey's weakness, if any, looks to be an out route

# Speed Metrics:

- MPH
- Acceleration
- Yards/frame

We will take the top 2 best values for each speed metric, of each player in each game, then average them out to get a good gauge on how fast/quick they are. Grabbing the top 2 speeds is to ensure there are no outliers in the data, I did find erroneous tracking speeds in gameid 2018102101

In [ ]:
Speed = Finaldf.query('displayName != "Football" & gameId != 2018102101 & position == "CB"').groupby(['nflId','week','gameId','playId'])['s','a','dis'].max().reset_index()
Speed['MPH'] = Speed['s'] / 0.488889
Speed1 = Speed.groupby(['nflId','week','gameId'])['MPH'].apply(lambda x: x.nlargest(2).mean()).reset_index()
Speed1['a'] = Speed.groupby(['nflId','week','gameId'])['a'].apply(lambda x: x.nlargest(2).mean()).reset_index()['a']
Speed1['dis'] = Speed.groupby(['nflId','week','gameId'])['dis'].apply(lambda x: x.nlargest(2).mean()).reset_index()['dis']
Speed1 = Speed1.groupby(['nflId'])['MPH','a','dis'].mean().reset_index()
Speed1.sort_values(by=['MPH'], ascending=False).head(10)

In [ ]:
Speed1 = Speed1.set_index('nflId')
Speed1 = Speed1.loc[~Speed1.index.duplicated(keep='first')]
Final2["Speed"] = Final2.nflId.map(Speed1['MPH'])
Final2["Acceleration"] = Final2.nflId.map(Speed1['a'])
Final2["yds/frame"] = Final2.nflId.map(Speed1['dis'])
Final2.head(5)

# Speed Vs. Acceleration

In [ ]:
fig = px.scatter(Final2, x="Speed", y="Acceleration", 
                 color="Name",
                 width=800, height=700,template='plotly_dark')
fig.show()

# Combine Metrics:

We are going to scrape https://www.pro-football-reference.com/ and get combine data for the following players. This will help us see if any of the metrics like 40 yard dash, 3 cone drill, shuttle, etc. are significant in player evaluation.

In [ ]:
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import re

def findTables(url):
    res = requests.get(url)
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.findAll('div', id = "content")
    divs = divs[0].findAll("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.findAll("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)

def pullTable(url, tableID, header = True):
    res = requests.get(url)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.findAll('table', id = tableID)
    data_rows = tables[0].findAll('tr')
    game_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
        for i in range(len(data_rows))
        ]
    data = pd.DataFrame(game_data)
    if header == True:
        data_header = tables[0].findAll('thead')
        data_header = data_header[0].findAll("tr")
        data_header = data_header[0].findAll("th")
        header = []
        for i in range(len(data.columns)):
            header.append(data_header[i].getText())
        data.columns = header
        data = data.loc[data[header[0]] != header[0]]
    data = data.reset_index(drop = True)
    return(data)


def PullData(df, start, end):
    Db_Names = df['Name'].to_list()
    Pos = ['CB','DB','FS','SS','S']
    combinedata = []
    Years = range(start, end+1)
    for yr in Years:
        print(yr, end=' ')
        URL = ('https://www.pro-football-reference.com/draft/' + str(yr) + '-combine.htm')
        id = 'combine'
        data1 = pullTable(URL, id, header=True)
        data1['Year'] = yr
        data1 = data1[(data1['Player'].isin(Db_Names)) & (data1['Pos'].isin(Pos)) ]
        combinedata.append(data1)
    final = pd.concat(combinedata)
    final.rename(columns = {"Player": "Name"},inplace = True) 
    final = final.set_index('Name')
    final = final.loc[~final.index.duplicated(keep='first')]
    df["Ht"] = df.Name.map(final['Ht']).apply(lambda x: np.nan if pd.isnull(x) else 12*int(x.split('-')[0])+int(x.split('-')[1]))
    df['Wt'] = df.Name.map(final['Wt'].astype(float))
    df['40yd'] = df.Name.map(final['40yd'].astype(float))
    df['Vertical'] = df.Name.map(final['Vertical'])
    df['Bench'] = df.Name.map(final['Bench'])
    df['Broad Jump'] = df.Name.map(final['Broad Jump'])
    df['3Cone'] = df.Name.map(final['3Cone'])
    df['Shuttle'] = df.Name.map(final['Shuttle'])
    df['YearsInLeague'] = 2018 - df.Name.map(final['Year'].astype(int))
 #   df["Ht"] = np.where(df["Ht"].notnull() == True,df["Ht"].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1])),df["Ht"])
    return df


def change_names(x):
    return x.replace('0','Outside Corners').replace('1','Slot Corners').replace('2',"Hybrid Corners")

Some players have no combine data for some events, or none at all. So we will fill those in with the cluster median for each feature

In [ ]:
data = PullData(Final2,2000, 2019)
data['Cluster_Rank'] = data.groupby(['Cluster'])['Rank'].rank(ascending=True)



data = data.replace(r'^\s*$', np.nan, regex=True)

data['40yd'] = data['40yd'].astype(float)
data['Vertical'] = data['Vertical'].astype(float)
data['Bench'] = data['Bench'].astype(float)
data['Broad Jump'] = data['Broad Jump'].astype(float)
data['3Cone'] = data['3Cone'].astype(float)
data['Shuttle'] = data['Shuttle'].astype(float)


data['Tier'] = np.where(data['Cluster_Rank'] <= 10,"Top Ten","Middle")
data['Worst_ClustRank'] = data.groupby(['Cluster'])['Cluster_Rank'].transform('max')
data['Tier'] = np.where(data['Cluster_Rank'] / data['Worst_ClustRank'] > .70,"Worst",data['Tier'])

data['Ht'] = data['Ht'].fillna(data.groupby(['Cluster'])['Ht'].transform('median'))
data['Wt'] = data['Wt'].fillna(data.groupby(['Cluster'])['Wt'].transform('median'))
data['40yd'] = data['40yd'].fillna(data.groupby(['Cluster'])['40yd'].transform('median'))
data['Vertical'] = data['Vertical'].fillna(data.groupby(['Cluster'])['Vertical'].transform('median'))
data['Bench'] = data['Bench'].fillna(data.groupby(['Cluster'])['Bench'].transform('median'))
data['Broad Jump'] = data['Broad Jump'].fillna(data.groupby(['Cluster'])['Broad Jump'].transform('median'))
data['3Cone'] = data['3Cone'].fillna(data.groupby(['Cluster'])['3Cone'].transform('median'))
data['Shuttle'] = data['Shuttle'].fillna(data.groupby(['Cluster'])['Shuttle'].transform('median'))
data['YearsInLeague'] = data['YearsInLeague'].fillna(data.groupby(['Cluster'])['YearsInLeague'].transform('median'))
data['Cluster'] = data['Cluster'].astype(str).map(lambda x: change_names(x))
data

In [ ]:
!pip install joypy

In [ ]:
def CreatePlots(df, feature):
    import joypy
    %matplotlib inline

    df["Top Ten Tier"] = df.apply(lambda row: row[feature] if row["Tier"] == "Top Ten" else np.nan, axis = 1)
    df["Middle Tier"] = df.apply(lambda row: row[feature] if row["Tier"] == "Middle" else np.nan, axis = 1)
    df["Worst Tier"] = df.apply(lambda row: row[feature] if row["Tier"] == "Worst" else np.nan, axis = 1)


    plt.figure(dpi = 4000)
    # plot the data using joypy
    fig, axes = joypy.joyplot(data, 
                            column=['Top Ten Tier', 'Middle Tier', 'Worst Tier'], # colums to be plotted.
                            by = "Cluster", # separate the data by this value. Creates a separate distribution for each one.
                            ylim = 'own', 
                            figsize = (10,7), 
                            legend = True, 
                            color = [ '#66ffcc','#4d94ff', '#ff9966'],
                            background='white',
                            alpha = 0.6, linecolor="k",linewidth=3,ylabels=True,xlabels=True,
                            )

    plt.title(str(feature) + ' distribution of Tiers by the Cluster', fontsize = 35, color = 'Black', alpha = 1)
    plt.rc("font", size = 20)
    plt.xlabel(str(feature) + ' of Player by Cluster',  fontsize = 16, color = 'Black', alpha = 1)
    plt.ylabel('Cluster',  fontsize = 16, color = 'Black', alpha = 1);
    plt.show()

# Tier Density Plots:

Now we will for loop through meaningful features, groupby cluster, and compare the ranking tiers via density plots

In [ ]:
Cols = ['Avg_Separation', 'Avg_EPA', 'Comp_pct', 'AvgYards_Allowed', 'GroundCovered_Avg', 'Speed', 'Acceleration', 'yds/frame', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'YearsInLeague']

for col in Cols:
    CreatePlots(data, col)

# Final Thoughts:

If we focus solely on Top Tier outside corners, based on the plots above, it appears that the elite group may have a slight speed and size advantage. Which makes complete sense, size gives them the ability to contest catches or be more physical at the line of scrimmage and speed allows them to run stride for stride next to any receiver. 

A decent amount of the elite cover corners consistently hit near 19 mph on average. What I found interesting was the 40 yard dash distributions didn't really align with the MPH distribution, are track speed and game speed really two different things?

Also, the 3 cone drill and shuttle are combine tests to measure quickness and change of direction, but in each cluster group you could see that there was hardly any difference between tiers.


# Who will find this submission useful:

- Fantasy sites / private companies that want to use tracking data as a means to gather shadow coverage statistics. I know PFF tracks shadow stats, but their method of doing so could be having to physically watch each individual play which is time consuming, costly and prone to human error. I think the whole process could be automated or semi-automated.

- Inspiring Fantasy sites / private companies to look at new ways of feature engineering new variables that their end users will find useful

- NFL Analytics staffers may gain some insight or inspiration from this submission

- A curious NFL fan

## Other Work:

1. [Shadow Cornerback + Coverage Analysis](https://www.kaggle.com/jdruzzi/shadow-cornerback-coverage-analysis)

2. [Defender Bite Velocity on Play-Action](https://www.kaggle.com/jdruzzi/defender-bite-velocity-on-play-action)

3. [Pass Coverage Classification](https://www.kaggle.com/jdruzzi/pass-coverage-classification-80-recall)

4. [Quantifying Press Coverage](https://www.kaggle.com/jdruzzi/quantifying-press-coverage-ability)

5. [Defender Tendencies: One-Cut Routes + Double Moves](https://www.kaggle.com/jdruzzi/defender-tendencies-one-cut-routes-double-moves)

## Data:

[Revised BDB Data](https://www.kaggle.com/jdruzzi/revised-bdb-data)